In [1]:
!git clone https://github.com/xpuria/glimpse-mds.git

Cloning into 'glimpse-mds'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 432 (delta 89), reused 58 (delta 58), pack-reused 326 (from 3)
Receiving objects: 100% (432/432), 32.17 MiB | 22.50 MiB/s, done.
Resolving deltas: 100% (275/275), done.


In [2]:
cd glimpse-mds

/content/glimpse-mds


In [3]:
!python glimpse/data_loading/data_processing.py --dataset_path /content/glimpse-mds/data/all_reviews_2017.csv

/content/glimpse-mds/glimpse/data_loading/data_processing.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_dataset.rename(columns={"review": "text", "metareview": "gold"}, inplace=True)
/content/glimpse-mds/glimpse/data_loading/data_processing.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_dataset.rename(columns={"review": "text", "metareview": "gold"}, inplace=True)
/content/glimpse-mds/glimpse/data_loading/data_processing.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [4]:
pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [5]:
ls

 aspect-based-summarization.csv                 'multimodels_summarization (1).ipynb'
'Aspect_Summerizer_final (1).ipynb'              multi_model_summarization.csv
 data/                                           pyproject.toml
 examples/                                       Readme.md
 glimpse/                                        requirements
'Glimspe_Hybrid_Summarization_Final (1).ipynb'   rsasumm/
 hybrid-summarization                            scripts/
 mds/


In [7]:
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from bert_score import score as bertscore_score

# Import your original RSA reranking implementation.
# (Ensure that the module rsasumm is in your Colab path.)
from rsasumm.rsa_reranker import RSAReranking

# ------------------------------------------
# Adaptive RSA Reranking Extension
# ------------------------------------------
class AdaptiveRSAReranking(RSAReranking):
    # Override compute_conditionned_likelihood to pass a maximum length.
    def compute_conditionned_likelihood(self, x, y, mean=True):
        loss_fn = torch.nn.CrossEntropyLoss(reduction="none")
        # Force truncation to the model's max length (1024 for BART-large-cnn)
        x_enc = self.tokenizer(
            x,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(self.device)
        y_enc = self.tokenizer(
            y,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(self.device)
        outputs = self.model(
            input_ids=x_enc.input_ids,
            decoder_input_ids=y_enc.input_ids,
            attention_mask=x_enc.attention_mask,
            decoder_attention_mask=y_enc.attention_mask,
        )
        logits = outputs.logits
        # Shift logits and target tokens for next-token prediction
        shifted_logits = logits[..., :-1, :].contiguous()
        shifted_ids = y_enc.input_ids[..., 1:].contiguous()
        loss = loss_fn(shifted_logits.view(-1, shifted_logits.size(-1)), shifted_ids.view(-1))
        loss = loss.view(len(x), -1).sum(-1)
        if mean:
            non_pad = (y_enc.input_ids != self.tokenizer.pad_token_id).float().sum(-1)
            loss = loss / non_pad
        # Return negative loss as likelihood (higher is better)
        return -loss

    def adaptive_rerank(self, max_iter=10, epsilon=1e-3, alpha=0.5):
        """
        Adaptive RSA reranking extension that:
         - Iterates the RSA calculations until convergence.
         - Selects the best candidate summary based solely on RSA probabilities.
         - Optionally combines RSA with BERTScore F1 (weighted by alpha).

        Parameters:
          - max_iter: Maximum number of RSA iterations.
          - epsilon: Convergence threshold (mean absolute difference).
          - alpha: Weight for RSA score (with 1 - alpha for BERTScore).

        Returns a dictionary containing:
          - "best_rsa": Best candidate (per source) based solely on RSA.
          - "best_combined": Best candidate based on combined RSA+BERTScore.
          - "rsa_probs": Final RSA probability matrix.
          - "bertscore": BERTScore F1 matrix.
          - "combined_score": Final combined score matrix.
        """
        # Compute the initial likelihood matrix using the imported RSAReranking method.
        self.initial_speaker_probas = self.likelihood_matrix()  # shape: (num_sources, num_candidates)

        # Initialize RSA distributions in log-space.
        S_prev = torch.log_softmax(self.initial_speaker_probas, dim=-1)
        L_prev = torch.log_softmax(self.initial_speaker_probas, dim=0)

        # Adaptive iteration: update S and L until convergence.
        for t in range(1, max_iter + 1):
            S_new = torch.log_softmax(self.rationality * L_prev, dim=-1)
            L_new = torch.log_softmax(S_new, dim=0)
            diff = torch.abs(torch.exp(S_new) - torch.exp(S_prev)).mean().item()
            print(f"Iteration {t}: mean difference = {diff:.6f}")
            if diff < epsilon:
                print(f"Convergence reached at iteration {t} (diff = {diff:.6f})")
                break
            S_prev, L_prev = S_new, L_new

        rsa_probs = torch.exp(S_new)  # Final RSA probabilities

        # Select best candidate based solely on RSA.
        best_rsa_indices = torch.argmax(rsa_probs, dim=-1)
        best_rsa = [self.candidates[i] for i in best_rsa_indices.cpu().tolist()]

        # Compute BERTScore F1 for each candidate vs. each source text.
        num_sources = len(self.source_texts)
        num_candidates = len(self.candidates)
        bertscore_matrix = torch.zeros((num_sources, num_candidates), device=self.device)
        for j, candidate in enumerate(self.candidates):
            candidates_list = [candidate] * num_sources
            references_list = self.source_texts
            # Compute BERTScore (F1 scores)
            _, _, F1 = bertscore_score(candidates_list, references_list, lang="en", verbose=False)
            bertscore_matrix[:, j] = F1.to(self.device)

        # Combine RSA probabilities and BERTScore (both assumed to be in [0, 1])
        combined_score = alpha * rsa_probs + (1 - alpha) * bertscore_matrix
        best_combined_indices = torch.argmax(combined_score, dim=-1)
        best_combined = [self.candidates[i] for i in best_combined_indices.cpu().tolist()]

        return {
            "best_rsa": best_rsa,
            "best_combined": best_combined,
            "rsa_probs": rsa_probs,
            "bertscore": bertscore_matrix,
            "combined_score": combined_score
        }

# ------------------------------------------
# Candidate Summary Generation using BART
# ------------------------------------------
def generate_candidate_summaries(model, tokenizer, text, num_candidates=8, max_length=100):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="longest",
        max_length=1024
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_candidates,
        do_sample=True,
        top_p=0.95,
        early_stopping=True
    )
    candidates = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]
    return candidates

# ------------------------------------------
# Main Pipeline (Variables set for Colab)
# ------------------------------------------
def main():
    # Set variables directly (since we are not using argparse in Colab)
    input_csv = "/content/glimpse-mds/data/processed/all_reviews_2017.csv"       # Path to your input CSV (upload to Colab)
    output_csv = "adaptive_summarization.csv"     # Path to save results
    num_candidates = 4           # Number of candidate summaries per source
    max_length = 100              # Maximum summary length
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Read the CSV file.
    df = pd.read_csv(input_csv)
    df = df.iloc[:314]
    # Expected CSV columns: id, text, gold
    # For example:
    # id                                      text                                   gold
    # https://openreview.net/forum?id=...      Summary: The paper presents ...        The program committee appreciates...
    # https://openreview.net/forum?id=...      Results on the VQA task are good ...   The program committee appreciates...

    # Group rows by id: concatenate all texts and take the first gold summary.
    grouped = df.groupby("id").agg({
        "text": lambda texts: " ".join(texts),
        "gold": "first"
    }).reset_index()
    print(f"Found {len(grouped)} unique ids.")

    # Load BART model and tokenizer (facebook/bart-large-cnn)
    model_name = "facebook/bart-large-cnn"
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    results_list = []
    # Process each unique id.
    for _, row in grouped.iterrows():
        source_id = row["id"]
        source_text = row["text"]
        gold_summary = row["gold"]
        print(f"Processing id: {source_id}")

        # Generate candidate summaries.
        candidates = generate_candidate_summaries(
            model, tokenizer, source_text,
            num_candidates=num_candidates,
            max_length=max_length
        )
        print("Generated candidates:")
        for idx, cand in enumerate(candidates, start=1):
            print(f"  Candidate {idx}: {cand}")

        # Prepare for RSA reranking (using the concatenated source text as a one-element list).
        adaptive_reranker = AdaptiveRSAReranking(
            model=model,
            tokenizer=tokenizer,
            candidates=candidates,
            source_texts=[source_text],
            batch_size=8,
            rationality=3,  # Adjust as needed.
            device=device
        )
        rerank_results = adaptive_reranker.adaptive_rerank(max_iter=10, epsilon=1e-3, alpha=0.5)
        best_rsa = rerank_results["best_rsa"][0]         # Best candidate based solely on RSA.
        best_combined = rerank_results["best_combined"][0] # Best candidate based on combined RSA+BERTScore.

        result_entry = {
            "id": source_id,
            "source_text": source_text,
            "gold": gold_summary,
            "candidates": candidates,
            "best_rsa": best_rsa,
            "best_combined": best_combined,
            "rsa_probs": rerank_results["rsa_probs"].cpu().numpy().tolist(),
            "bertscore": rerank_results["bertscore"].cpu().numpy().tolist(),
            "combined_score": rerank_results["combined_score"].cpu().numpy().tolist()
        }
        results_list.append(result_entry)

    # Save results to an output CSV.
    output_df = pd.DataFrame(results_list)
    output_df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Run the main pipeline
main()

Found 103 unique ids.
Processing id: https://openreview.net/forum?id=B1TTpYKgx
Generated candidates:
  Candidate 1: This paper presents a theoretical and empirical approach to the problem of understanding the expressivity of deep networks. Random networks (deep networks with random Gaussian weights, hard tanh or ReLU activation) are studied. The paper is too obscure and too long. The work may have some interesting ideas but it does not seem to be properly replaced in context.
  Candidate 2: This paper presents a theoretical and empirical approach to the problem of understanding the expressivity of deep networks. Random networks (deep networks with random Gaussian weights, hard tanh or ReLU activation) are studied according to several criterions. There doesn't seem to be a solid justification for why the newly introduced measures of expressivity really measure expressivity.
  Candidate 3: This paper presents a theoretical and empirical approach to the problem of understanding the expres

100%|██████████| 1/1 [00:07<00:00,  7.98s/it]


Iteration 1: mean difference = 0.173798
Iteration 2: mean difference = 0.000000
Convergence reached at iteration 2 (diff = 0.000000)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Processing id: https://openreview.net/forum?id=B1jnyXXJx
Generated candidates:
  Candidate 1: This paper proposes a novel method for accelerating optimization near saddle points. The basic idea is to repel the current parameter vector from a running average of recent parameter values. This method is shown to optimize faster than a variety of other methods. It would be very valuable to think of a clear diagram illustrating how it differs from momentum.
  Candidate 2: This paper proposes a novel method for accelerating optimization near saddle points. The basic idea is to repel the current parameter vector from a running average of recent parameter values. This method is shown to optimize faster than a variety of other methods. Authors of the paper show results over several different datasets.
  Candidate 3: This paper proposes a novel method for accelerating optimization near saddle points. The basic idea is to repel the current parameter vector from a running average of recent paramete

...

Processing id: https://openreview.net/forum?id=ryuxYmvel
Generated candidates:
  Candidate 1: Use of ML in ITP is an interesting direction of research. Authors consider the problem of predicting whether a given statement would be useful in a proof of a conjecture or not. This is posed as a binary classification task and authors propose a dataset and some deep learning based baselines. The experimental results are impressively good for a first baseline.
  Candidate 2: Use of ML in ITP is an interesting direction of research. Authors consider the problem of predicting whether a given statement would be useful in a proof of a conjecture or not. This is posed as a binary classification task and authors propose a dataset and some deep learning based baselines.
  Candidate 3: Use of ML in ITP is an interesting direction of research. Authors consider the problem of predicting whether a given statement would be useful in a proof of a conjecture or not. This is posed as a binary classification 

100%|██████████| 1/1 [00:07<00:00,  7.16s/it]


Iteration 1: mean difference = 0.291686
Iteration 2: mean difference = 0.000000
Convergence reached at iteration 2 (diff = 0.000000)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Results saved to adaptive_summarization.csv
